# SYSC 535: FINAL  PROJECT
## Agent Based Simulation of Bottleneck Free Traffic Jam 
### Christian Hower
### Spring 2019



Traffic has a major impact on the quality of people’s lives. It is always frustrating to be stuck in stop and go traffic but it is especially frustrating when there is seemingly no need for the traffic. The phenomenon of stop and go traffic that occurs when there is sufficient roadway to handle the volume of cars at a given time is called bottleneck free traffic. This has been studied experimentally via placing actuals drivers on a circular track. It has been shown that simple fluctuations in accelerating and braking can be sufficient to cause the wave behavior observed in real traffic conditions.

This project recreates the dynamics of bottleneck free traffic using an agent-based simulation. The agents move in a loop and the next position of the agent is evaluated locally. Each agent tries to move at some constant speed in order to prevent traffic build up on the track. There are (2) behaviors that model different driving assumptions: 'Constant', which models a perfect driver that always moves the target distance at each time interval that the all agents are trying to move. This type of agent could be a good model for autonomous vehicles. 'Constant random' is the second behavior which models imperfect distances travelled at each time interval via sampling from a gaussian distribution. When the mix argument is set to False, all agents are Human agents. When the mix argument is set to True, the population is approximately 50/50 humans and autonomous vehicles.


In [35]:
import numpy as np
import pandas as pd

In [36]:
class agent():
    def __init__(self):
        self.type = np.random.randint(0,2)
        self.rear_neighbor_position = None
        self.front_neighbor_position = None
        self.target = None

In [37]:
class traffic_sim:
# Intitialization functions
    def __init__(self, n_agents, resolution = 6, mix = False):
        self.n_spaces = n_agents * resolution # car lengths
        self.resolution = resolution
        self.n_agents = n_agents # number of cars on the track
        self.agents = []
        self.iterations = 0
        self.behavior = mix
        hist_columns = ['iteration','env']   
        self.history = pd.DataFrame(columns = [hist_columns])
     
        self.__populate__()
        self.__observe_positions__()
        
    def __populate__(self):
        initial_positions = np.arange(0,self.n_spaces, self.resolution)       
        c = self.n_agents - 1
        
        for i in range(self.n_agents):
            ag = agent()
            ag.position = initial_positions[i]
            ag.rear_neighbor_position = initial_positions[i-1]
            ag.front_neighbor_position= initial_positions[i-c]
            ag.id = i + 1
            if self.behavior == True:
                ag.type = np.random.randint(0,2)
            else:
                ag.type = 0
            self.agents.append(ag)
            
            
# Update functions
    def __update_positions__(self):
        for ag in self.agents:
            if ag.type == 1:                
                self.__constant__(ag)
            else:                
                self.__constant_target__(ag)
            
        self.__observe_positions__()


    def __constant__(self, ag): 
        ag.position += self.n_agents
        if ag.position > self.n_spaces - 1:
            ag.position = ag.position - self.n_spaces
 
        
    def __constant_target__(self, ag):       
        target = ag.position + self.n_agents
        if target > self.n_spaces - 1:
            target = target - self.n_spaces
        
        ag.position = np.random.normal(target,1)//1
        if ag.position > self.n_spaces - 1:
            ag.position = ag.position - self.n_spaces
                
# Observation functions
        
    def __observe_positions__(self):
        self.__get_neighbor_positions__()
        env = self.__get_env__()
        self.history = self.history.append({'iteration': self.iterations ,
                                            'env': env}, ignore_index=True)
        
        self.iterations += 1 

        
    def __get_neighbor_positions__(self):
        curr_positions = []
        c = int(self.n_agents - 1)
        
        for ag in self.agents:
            curr_positions.append(ag.position)
        curr_positions = np.asarray(curr_positions)
        
        for i in range(self.n_agents):
            self.agents[i].rear_neighbor_position = curr_positions[i-1]
            self.agents[i].front_neighbor_position = curr_positions[i-c]
        
        
    def __get_env__(self):
        env = np.zeros(self.n_spaces, dtype = int)
        curr_positions = []
        
        for ag in self.agents:
            curr_positions.append((ag.position, ag.id))


        for pos in curr_positions:
            a , b = pos
            env[int(a)] = b
        
        return env

# Run simulation functions    
    def run_sim(self, sims):        
        for sim in range(sims):
            
            self.__update_positions__()

            for ag in self.agents:
                if ag.position == ag.front_neighbor_position:
                    return self.iterations
           
            if sim == sims - 1:
                return self.history               

    def view_sim(self):
        for sim in range(len(self.history)):
            print(self.history['env'][sim])

# Helper functions
    def print_attr(self):
        for ag in self.agents:
            print('id:', ag.id, 'position:', ag.position, 'rear neighbor', ag.rear_neighbor_position,
                  'front neighbor', ag.front_neighbor_position, 'target', ag.target)

### Humans Only

In [41]:
ts = traffic_sim(6, mix = False)
ts.run_sim(100)
ts.view_sim()

[1 0 0 0 0 0 2 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 0 5 0 0 0 0 0 6 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 3 0 0 0 0 0 0 0 4 0 0 0 0 5 0 0 0 0 0 6]
[0 0 0 0 0 6 0 0 0 0 0 0 1 0 0 0 0 0 0 2 0 0 3 0 0 0 0 0 0 4 0 0 0 5 0 0]
[0 0 5 0 0 0 0 0 0 0 0 6 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 3 0 0 0 0 0 0 4]
[0 0 0 4 0 0 0 0 5 0 0 0 0 0 0 0 0 6 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 3 0 0]
[0 2 0 3 0 0 0 0 4 0 0 0 0 0 0 0 5 0 0 0 0 0 6 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 2 0 0 3 0 0 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 0 0 6 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 0 0 2 3 0 0 0 0 0 0 4 0 0 0 5 0 0 0 0 0 0 0 0 0 6]
[0 0 0 0 6 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 5 0 0 0 0]


#### Humans Only, No robots allowed
Congestion and pile ups can be seen in with the constant target behavior that characterizes human drivers. Although the agents start uniformly distributed on the track, traffic conditions appear after only a few time steps. A key feature of traffic jams is the oscillating pattern that looks something like peristalsis. This is difficult to quantify but qualitatively, there does seem to be some similarity in the results of the simulation. Congestion builds up and often persists for multiple time steps. 

### Humans and Autonomous Vehicles 

In [47]:
ts = traffic_sim(6 , mix = True)
ts.run_sim(100)
ts.view_sim()

[1 0 0 0 0 0 2 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 0 5 0 0 0 0 0 6 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 0 5 0 0 0 0 6]
[5 0 0 0 0 6 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 3 0 0 0 0 0 4 0 0 0 0 0]
[4 0 0 0 0 0 5 0 0 0 0 6 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 3 0 0 0 0 0]
[3 0 0 0 0 0 4 0 0 0 0 0 5 0 0 0 0 0 6 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0]
[2 0 0 0 0 0 3 0 0 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 6 0 0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 2 0 0 0 0 0 3 0 0 0 0 0 0 4 0 0 0 0 5 0 0 0 0 6 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 3 0 0 0 0 0 0 4 0 0 0 0 5 0 0 0 0 6]
[5 0 0 0 6 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 3 0 0 0 4 0 0 0 0 0 0 0]
[0 0 0 0 0 0 5 0 0 6 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 3 0 4 0 0 0]
[3 0 0 4 0 0 0 0 0 0 0 0 5 0 0 0 6 0 0 0 0 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 3 0 4 0 0 0 0 0 0 0 0 0 5 0 6 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 3 4 0 0 0 0 0 0 0 0 0 0 5 0 6 0 0 0 0 0 0 0 0 2]
[0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 0 0 4

#### Humans and Autonomous Vehicles
When the mix argument is set to true, about half the population of agents will always meet their target position in the next time step. The goal here is to see if having some number of perfect agents will prevent the buildup of traffic. Although human drivers that make errors that build up over time, it is possible that some number of perfect agents will prevent the system from going into some critical state that leads to traffic that cannot be recovered from.


#### Future Work/ To Do
While this simulation does capture some of the mechanics of the real system, it does so in a fairly abstract way and in a way that is hard to quantity. I had considered using some measure of squared distance between neighbors to calculate aggregate level of traffic for each time step but this turned out to be difficult to implement. With more time, I would like to devise a metric for the overall traffic amount and find a way to better visualize the simulation for easier qualitative assessment.

